In [1]:
%load_ext autoreload
%autoreload 2
main_dir = r'../../..'
data_path = main_dir+'/data'
import sys
sys.path.append(main_dir)
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union

from BayDS.lib.pipeline import *
from BayDS.lib.io import *
import os


In [2]:
data_dir =f'{main_dir}/Snapshots/1/05-LabelEncoded-last'
p = Pipeline(working_folder=f'{main_dir}/Snapshots/1/FindUselessLast')

p.add_node(LoaderNode, None, 'data',
           params={
               'input_directory': data_dir,
               'file': 'label_encoded_data.pkl'
           })
p.add_node(LoaderNode, None, 'numerical_columns',
           params={
               'input_directory': data_dir,
               'file': 'numerical_columns.yaml'
           })
p.add_node(LoaderNode, None, 'categorical_columns',
           params={
               'input_directory': data_dir,
               'file': 'categorical_columns.yaml'
           })
p.add_node(LoaderNode, None, 'bad_for_training_features',
           params={
               'input_directory': data_dir,
               'file': 'bad_for_training_features.yaml'
           })
p.save()
p.run(verbose=True)

bad_features = p.data['bad_for_training_features']

if 'isFraud' in bad_features:
    bad_features.remove('isFraud')

p.save()
p.run(verbose=True)

---------------------------
0: LoaderNode [2019-09-29 23:21:52]
params:
 {'input_directory': '../../../Snapshots/1/05-LabelEncoded-last', 'file': 'label_encoded_data.pkl'}
---------------------------
1: LoaderNode [2019-09-29 23:22:01]
params:
 {'input_directory': '../../../Snapshots/1/05-LabelEncoded-last', 'file': 'numerical_columns.yaml'}
---------------------------
2: LoaderNode [2019-09-29 23:22:01]
params:
 {'input_directory': '../../../Snapshots/1/05-LabelEncoded-last', 'file': 'categorical_columns.yaml'}
---------------------------
3: LoaderNode [2019-09-29 23:22:01]
params:
 {'input_directory': '../../../Snapshots/1/05-LabelEncoded-last', 'file': 'bad_for_training_features.yaml'}


In [3]:
p.add_node(DropFeaturesNode, 
           ('data', 'numerical_columns','categorical_columns'),
           params={
               'drop': bad_features,
           })

p.save()
p.run(verbose=True)

---------------------------
4: DropFeaturesNode [2019-09-29 23:22:01]
params:
 {'drop': ['BrowserAge', 'D8_to_mean_groupby_uid', 'D8_to_mean_groupby_uid2', 'D8_to_mean_groupby_uid3', 'D8_to_mean_groupby_uid4', 'D8_to_mean_groupby_uid5', 'D1_to_mean_groupby_new_card_id', 'D1_to_stdscore_groupby_new_card_id', 'D2_to_mean_groupby_new_card_id', 'D2_to_stdscore_groupby_new_card_id', 'D8_to_mean_groupby_new_card_id', 'D8_to_mean_groupby_bank_type', 'Date', 'TransactionDT', 'DT_M', 'DT_W', 'DT_D', 'Transaction_freq_5_past', 'Transaction_freq_10_past', 'Transaction_freq_100_past']}


In [4]:
p.add_node(DownsamplingTrainNode, 'data', 'data',
          params={
              'strategy':'reduction_factor',
              'reduction_factor':0.2
          })
p.save()
p.run(verbose=True)

---------------------------
5: DownsamplingTrainNode [2019-09-29 23:22:14]
params:
 {'strategy': 'reduction_factor', 'reduction_factor': 0.2}


## Code to make pipeline node

In [5]:
df = p.data['data']

In [6]:
from sklearn.feature_selection import RFECV
train_ids = df[df.isFraud >= 0].index
X = df.loc[train_ids].drop('isFraud',axis=1)
y = df.loc[train_ids,'isFraud']
X.replace(np.inf, np.nan, inplace=True)
X.replace(-np.inf, np.nan, inplace=True)
X.fillna(-999, inplace=True)

In [7]:
for col in X.columns:
    if str(X[col].dtype) == 'datetime64[ns]':
        print (col, X[col].dtype)
        X[col] = pd.to_timedelta(X[col]).map(lambda qq:qq.days)
#         X[col] = X[col].astype(datetime.datetime)

Date datetime64[ns]


c:\python37\lib\site-packages\pandas\util\_decorators.py:208: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  return func(*args, **kwargs)


start_date datetime64[ns]


In [8]:
params = {'num_leaves': 100,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': 8,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 0,
          'n_jobs': -1,
          'n_estimators':500
         }

In [9]:
import lightgbm as lgb 
from sklearn.model_selection import KFold
clf = lgb.LGBMClassifier(**params)
rfe = RFECV(estimator=clf, step=10, cv=KFold(n_splits=2, shuffle=False), scoring='roc_auc', verbose=2)

In [10]:
rfe.fit(X, y)

Fitting estimator with 947 features.
Fitting estimator with 937 features.
Fitting estimator with 927 features.
Fitting estimator with 917 features.
Fitting estimator with 907 features.
Fitting estimator with 897 features.
Fitting estimator with 887 features.
Fitting estimator with 877 features.
Fitting estimator with 867 features.
Fitting estimator with 857 features.
Fitting estimator with 847 features.
Fitting estimator with 837 features.
Fitting estimator with 827 features.
Fitting estimator with 817 features.
Fitting estimator with 807 features.
Fitting estimator with 797 features.
Fitting estimator with 787 features.
Fitting estimator with 777 features.
Fitting estimator with 767 features.
Fitting estimator with 757 features.
Fitting estimator with 747 features.
Fitting estimator with 737 features.
Fitting estimator with 727 features.
Fitting estimator with 717 features.
Fitting estimator with 707 features.
Fitting estimator with 697 features.
Fitting estimator with 687 features.
F

OSError: exception: access violation reading 0x0000000000000278

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
plt.figure(figsize=(9,4))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score")
plt.plot(range(1, len(rfe.grid_scores_) + 1), rfe.grid_scores_)


In [ ]:
rfe.ranking_ == 1
for i,col in enumerate(X.columns):
    if rfe.ranking_[i] == 1:
        print(col,rfe.ranking_[i])

In [ ]:
ranking = {}
for i,col in enumerate(X.columns):
        ranking[col] = int(rfe.ranking_[i])

In [ ]:
ranking

In [ ]:
import yaml

yaml.dump(ranking, open(p.working_folder+'/rfe.yaml','w'))